# `TorchDynamo`
## and (Maybe) PEP-523

Dboy Liao

## `TorchDynamo`

- What?
   - What is `TorchDynamo`?
   - What cat it do?

- Why?

- (Maybe) How?
    - [PEP-523](https://peps.python.org/pep-0523/)

## `TorchDynamo`: JIT Compiler Framework

- The workhorse for `torch.compile` under the hook
    - A Python-to-Python JIT compiler

![torchdynamo-jit](img/torchdynamo_jit.drawio.svg)

In [1]:
import torch

In [2]:
# A toy example
def foo(a: torch.Tensor, b: torch.Tensor):
    return torch.add(a, b)

# foo_ is compiled version of foo (JIT-enabled)
foo_ = torch.compile(foo)

In [3]:
a = torch.randn(3, 4)
b = torch.randn(3, 4)

In [4]:
%time
foo(a, b)

CPU times: user 7 μs, sys: 1 μs, total: 8 μs
Wall time: 14.5 μs


tensor([[ 2.1325, -1.3003,  0.0151, -1.1431],
        [-0.7481, -0.5271, -0.2793,  0.3850],
        [-0.2697,  0.3593, -2.1774,  0.4619]])

In [5]:
%time
foo_(a, b)

CPU times: user 6 μs, sys: 1 μs, total: 7 μs
Wall time: 14.1 μs


tensor([[ 2.1325, -1.3003,  0.0151, -1.1431],
        [-0.7481, -0.5271, -0.2793,  0.3850],
        [-0.2697,  0.3593, -2.1774,  0.4619]])

In [6]:
%time
foo_(a, b)

CPU times: user 7 μs, sys: 1 μs, total: 8 μs
Wall time: 15.5 μs


tensor([[ 2.1325, -1.3003,  0.0151, -1.1431],
        [-0.7481, -0.5271, -0.2793,  0.3850],
        [-0.2697,  0.3593, -2.1774,  0.4619]])

In [7]:
from depyf import decompile
from dis import dis
from torch._dynamo.eval_frame import _debug_get_cache_entry_list, innermost_fn

/home/dboy/open_source/play_torch.fx/.venv/lib/python3.11/site-packages/depyf/__init__.py:24: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/dboy/open_source/play_torch.fx/.venv/lib/python3.11/site-packages/depyf/VERSION.txt' mode='r' encoding='UTF-8'>
  __version__ = open(f"{os.path.dirname(__file__)}/VERSION.txt").read().strip()


In [8]:
cache_entries = _debug_get_cache_entry_list(innermost_fn(foo_))
cache_entry = cache_entries[0]
guard, code = cache_entry.check_fn, cache_entry.code

In [9]:
print(decompile(foo.__code__))

def foo(a, b):
    return torch.add(a, b)



In [10]:
print(decompile(code))

def foo(a, b):
    __temp_3, = __compiled_fn_1(b, a)
    return __temp_3



In [11]:
fn = innermost_fn(__compiled_fn_1)

In [12]:
print(fn.__closure__[0].cell_contents.__closure__[0].cell_contents.__closure__[0].cell_contents.__closure__[0].cell_contents.source_code)

# AOT ID: ['0_inference']
from ctypes import c_void_p, c_long, c_int
import torch
import math
import random
import os
import tempfile
from math import inf, nan
from torch._inductor.hooks import run_intermediate_hooks
from torch._inductor.utils import maybe_profile
from torch._inductor.codegen.memory_planning import _align as align
from torch import device, empty_strided
from torch._inductor.async_compile import AsyncCompile
from torch._inductor.select_algorithm import extern_kernels
from torch._inductor.codegen.multi_kernel import MultiKernelCall

aten = torch.ops.aten
inductor_ops = torch.ops.inductor
_quantized = torch.ops._quantized
assert_size_stride = torch._C._dynamo.guards.assert_size_stride
empty_strided_cpu = torch._C._dynamo.guards._empty_strided_cpu
empty_strided_cuda = torch._C._dynamo.guards._empty_strided_cuda
empty_strided_xpu = torch._C._dynamo.guards._empty_strided_xpu
reinterpret_tensor = torch._C._dynamo.guards._reinterpret_tensor
alloc_from_pool = torch.ops.inductor